In [1]:
#enviroment
# !pip install spacy
# !python -m spacy download zh_core_web_sm


import openai
from openai import OpenAI
import os
import spacy

OPENAI_API_KEY = 'sk-proj-o9TONJi0MW2tSiDMhRkxT3BlbkFJkUr03XQ5IfUaxamV0e3k'


In [2]:
nlp = spacy.load("zh_core_web_sm")

glossary_db = {
    '黑光纤': 'dark fiber',
    '光纤': 'optical fiber',
}

def identify_entities(source_text):
    doc = nlp(source_text)
    entities = []
    translations = {}
    for ent in doc.ents:
        term = ent.text.lower()
        if term in glossary_db:
            translations[ent.text] = glossary_db[term]
    return translations

text = "光纤照上去变成黑光纤了"
translations = identify_entities(text)

print("Identified entities:", translations)

Identified entities: {'黑光纤': 'dark fiber'}


In [3]:
def translate_text(source_language, target_language, prompt):

    client = OpenAI(api_key= OPENAI_API_KEY,)
    print(prompt)
    response = client.chat.completions.create(
      messages=[{
            "role": "user",
            "content": prompt,
          }],
      model="gpt-3.5-turbo",)
    translation = response.choices[0].message.content.strip().split("\n")[0]
    return translation

In [24]:
# Few-shot context changing via user input
def edit_context(context, original_text, source_language, target_language, few_shot_examples):

  edited_context = f'''Translate "{original_text}" from the {source_language} language, into the {target_language} language
                      using the following context style: "{context}".
                      Here are some examples of the style being used: {few_shot_examples}
                      '''
  return translate_text(source_language, target_language, edited_context)


# Chinese to Chinese Example 
print("CHINESE TO ENGLISH EXAMPLE")

# Poetic Example
examples = '''"Light speeds away, in a blink it will fly."
              "Stars reach from afar, with light from eons bold."
           '''

print(edit_context("A poetic two line rhyming style", "光纤照上去变成黑光纤了", "zh", "en", examples), "\n\n\n")

# Confused and Inquisitive Example
examples = '''"How does it work so well when it's so unclear?"
              "Why did it begin so suddenly, what triggered it?"
            '''

print(edit_context("A confused and inquisitive style", "光纤照上去变成黑光纤了", "zh", "en", examples), "\n\n\n")



# English to Chinese Example 
print("ENGLISH TO CHINESE EXAMPLE")

# Poetic Example
examples = '''"光逝如飞，瞬息即逝。"
              "星光遥来，古光犹豪。"
           '''

print(edit_context("A poetic two line rhyming style", "Lose track of time I'm bugging", "en", "zh", examples), "\n\n\n")

# Confused and Inquisitive Example
examples = '''"当它如此不清楚时，它如何运作得这么好？"
              "为什么它开始得这么突然，是什么触发了它？"
            '''

print(edit_context("A confused and inquisitive style", "Lose track of time I'm bugging", "en", "zh", examples))


CHINESE TO ENGLISH EXAMPLE
Translate "光纤照上去变成黑光纤了" from the zh language, into the en language
                      using the following context style: "A poetic two line rhyming style".
                      Here are some examples of the style being used: "Light speeds away, in a blink it will fly."
              "Stars reach from afar, with light from eons bold."
           
                      


"Fiber optics gleaming, turned into darkness beaming." 



Translate "光纤照上去变成黑光纤了" from the zh language, into the en language
                      using the following context style: "A confused and inquisitive style".
                      Here are some examples of the style being used: "How does it work so well when it's so unclear?"
              "Why did it begin so suddenly, what triggered it?"
            
                      
How does the fiber optic turn into a black fiber optic when it's being shone on? 



ENGLISH TO CHINESE EXAMPLE
Translate "Lose track of time I'm bugging" from the en language, into the zh language
                      using the following context style: "A poetic two line rhyming style".
                      Here are some examples of the style being used: "光逝如飞，瞬息即逝。"
              "星光遥来，古光犹豪。"
           
                      
时间失去，我烦恼。 



Translate "Lose track of time I'm bugging" from the en language, into the zh language
                      usin

In [96]:

# Feeding the model (OpenAI API) the original translation meta descriptors and prompting for a new/rephrased/alternate translation
def rephrase(old_translation, original_source_text, source_language, target_language):

  # Get entites so they don't have to be passed, simplifying the function call. Can be changed to passing if more efficient
  terms = identify_entities(original_source_text)
  entities_joined = '. '.join([f"'{k}': '{v}'" for k, v in terms.items()])

  # Construct the prompt for rephrased/alternate translation
  prompt = f'''Please provide an alternative translation to and rephrase: "{old_translation}" 
          based on the term {entities_joined}.
          The original text was {original_source_text}.
          The source language is {source_language} and the target language is {target_language}.'''

  # Make the call to the model
  return translate_text(source_language, target_language, prompt)

# Get alternative translation for demo text
rephrase("The fiber optic turned into dark fiber when the light shone on it.", "光纤照上去变成黑光纤了", "zh", "en")

Please provide an alternative translation to and rephrase: "The fiber optic turned into dark fiber when the light shone on it." 
          based on the term '黑光纤': 'dark fiber'.
          The original text was 光纤照上去变成黑光纤了.
          The source language is zh and the target language is en.


'The fiber optic transformed into dark fiber as it was illuminated.'

In [49]:
# prompt with entity translation
def prompt_generator(text, source_language, target_language):
  terms = identify_entities(text)
  terms_translations = '. '.join([f"'{k}': '{v}'" for k, v in terms.items()])
  translations = f"Based on the trasnlation of following term: {terms_translations}."
  prompt = f"Translate the following text from {source_language} into {target_language}: {text}\n"
  if terms == {}:
    return prompt
  prompt = translations + prompt

  return prompt

prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
# prompt = prompt_generator('Let them pray', 'en', 'zh')
print(prompt)

Based on the trasnlation of following term: '黑光纤': 'dark fiber'.Translate the following text from zh into en: 光纤照上去变成黑光纤了



In [113]:
# Get user input
text = "Let them pray"
# source_language = input("Enter the source language: ")
# target_language = input("Enter the target language: ")
prompt = prompt_generator(text, 'en', 'zh')
translation = translate_text("en", "zh", prompt)
print(translation)


prompt = prompt_generator('让他们祈祷', 'zh', 'en')
translation = translate_text("zh", "en", prompt)
print(translation)


prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
translation = translate_text("zh", "en", prompt)
print(translation)

Translate the following text from en into zh: Let them pray

让他们祈祷
Translate the following text from zh into en: 让他们祈祷

Let them pray.
Based on the trasnlation of following term: '黑光纤': 'dark fiber'.Translate the following text from zh into en: 光纤照上去变成黑光纤了

The fiber optic cable turned into dark fiber when the light shines on it.


In [73]:
#test
prompt = prompt_generator('光纤照上去变成黑光纤了', 'zh', 'en')
translation = translate_text('en', 'zh', prompt)
print(translation)

Based on the trasnlation of following term: '黑光纤': 'dark fiber'.Translate the following text from zh into en: 光纤照上去变成黑光纤了

The fiber optic cable became dark fiber when the light shone on it.


In [ ]:
#Quality Estimation